In [418]:
import json
import random
import pandas as pd
import jsonlines
import warnings
import plotly.graph_objects as go

pd.set_option('display.max_rows', 100)
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

In [437]:
file_path = '/home/carolus/Documents/school/green_ia/data/05_data/05_valid.jsonl'
num_samples = 500

In [438]:
def sample_jsonl_file(file_path, num_samples):
    sample_lines = []

    with open(file_path, 'r') as file:
        line_count = 0
        for line in file:
            line_count += 1
            if len(sample_lines) < num_samples:
                sample_lines.append(line)
            else:
                idx = random.randint(0, line_count - 1)
                if idx < num_samples:
                    sample_lines[idx] = line

    data_list = [json.loads(line) for line in sample_lines]
    df = pd.DataFrame(data_list)
    return df

In [439]:
df = sample_jsonl_file(file_path, num_samples)

In [440]:
df.tail(60)

,groups,packaging,name,ecoscore_tags,ecoscore_score,countries,ingredients,categories,labels_note
440,0.444444,empty,lait demi ecreme,0.75,36.0,0.311475,empty,,0.000000
441,0.111111,empty,organic poppy seed dressing,0.75,34.0,1.000000,empty,,0.222222
442,0.777778,empty,sweet peach tea,0.50,43.0,1.000000,water sugar disaccharide addedsugar,,0.000000
443,0.000000,,joghurt erdbeer,0.50,51.0,0.333333,sugar disaccharide addedsugar,snacks,0.777778
444,0.222222,empty,"sałatka z makrelą, pikantna.",1.00,0.0,0.737430,water oilandfat sugar disaccharide salt addeds...,,0.000000
445,0.000000,empty,fondant menthe subtile,1.00,15.0,0.000000,sugar disaccharide addedsugar,,0.000000
446,0.000000,empty,"whole foods market, dark chocolate almonds",0.75,22.0,1.000000,oilandfat sugar disaccharide addedsugar,confectioneries sweetsnacks snacks,0.000000
447,0.000000,empty,croissant glaseado y relleno de crema de cacao...,0.25,68.0,0.874317,empty,sweetsnacks snacks,0.000000
448,0.666667,empty,jambon a l'ancienne,1.00,12.0,0.288136,empty,meats meatsandtheirproducts,0.000000
449,0.000000,empty,miel de mûrier,0.25,72.0,0.311475,empty,,0.000000


In [423]:
uniques_groups = df['groups'].unique()
uniques_groups_list = list(uniques_groups)
for value in uniques_groups_list:
    print(value)

nan
0.0
0.5555555556
0.6666666667
0.2222222222
0.3333333333
0.4444444444
1.0
0.1111111111
0.7777777778
0.7
0.5
0.3


In [424]:
uniques_name = df['name'].unique()
uniques_name_list = list(uniques_name)
for value in uniques_name_list:
    print(value)

drizzle topping
stat warq
filet de saumon bio - lidl
frozen spring roll
milk chocolate eggs with soft fondant center
bananas
none
tagliatelle
cocktail sauce
penne blancs bio italie
avocado oil
non gmo polyfloral honey
yogourt framboise bio
oeuf en chocolat
chewits
pencils strawberry flavour
fuet iberico
yoghurt framboises
jambon belge séché
gluten free super fudge brownie single serve mix
assiette petits pois veau macaroni
core nutrition bar
mirepoix blend onions, celery, carrots, mirepoix blend
фасоль натуральная красная
fried rice seasoning
melange à crêpe mince
market basket root beer
filetti di alici distesi in olio delicato
goat cheese
whole grain chicken breast nuggets
гюбек
carrot puffs
transfers
sauce apéro citron basilic
jabón glicerinado
œufs de lompe
notte rossa
sustenium magnesio e potassio
fresh churned butter, honey butter
matcha green fasting tea
ciobar
company sushi chef wasabi powder
protein bar, chocolate caramel pecan, minis
saucisson sec au poivre
cabillaud et écrev

In [425]:
uniques_countries = df['countries'].unique()
uniques_countries_list = list(uniques_countries)
for value in uniques_countries_list:
    print(value)

1.0
0.3114754098
0.3333333333
0.138121547
0.3152173913
0.4316939891
0.1584699454
0.2960893855
nan
0.2881355932
0.8743169399
0.9945355191
0.0765027322
0.0
0.7650273224
0.4347826087
0.9734042553
0.1256830601
0.3184357542
0.0218579235
0.0558659218
0.2945205479
0.320754717
0.4189944134
0.5355191257
0.9016393443
0.1223404255
0.6229508197
0.306010929
0.8510638298
0.3031914894
0.2544378698
0.4293478261
0.3015873016
0.7595628415
0.0451977401
0.847826087
0.4124293785
0.6557377049
0.3715846995
0.3369565217
0.8715083799
0.9680851064
0.4207650273
0.0327868852
0.7486338798
0.6440677966
0.9943502825
0.3081081081
0.4285714286
0.7446808511
0.8700564972
0.6576086957
0.9890710383
0.0744680851
0.2336956522
0.8684210526
0.4143646409
0.3405405405
0.9625
0.9944134078
0.3227513228
0.0740740741
0.2921348315
0.1542553191
0.9273743017
0.1630434783
0.8988095238
0.1396648045
0.4378378378
0.7431693989
0.3442622951
0.5300546448
0.2786885246
0.2553191489
0.5901639344
0.1147540984
0.737704918
0.2847682119
0.398907103

In [426]:
uniques_packaging = df['packaging'].unique()
uniques_packaging_list = list(uniques_packaging)
for value in uniques_packaging_list:
    print(value)

empty
en plastique
carton plastique

glas
plastique
frais
kunststoff
en métal verre
sachet plastique
plastic
plastique carton
bottle
en bouteille plastique
bag
verre
bouteille verre
en sachet plastique


In [427]:
uniques_ecoscore_tags = df['ecoscore_tags'].unique()
uniques_ecoscore_tags_list = list(uniques_ecoscore_tags)
for value in uniques_ecoscore_tags_list:
    print(value)

nan
0.25
0.0
0.75
1.0
0.5


In [428]:
uniques_ecoscore_score = df['ecoscore_score'].unique()
uniques_ecoscore_score_list = list(uniques_ecoscore_score)
for value in uniques_ecoscore_score_list:
    print(value)

49.0
69.0
97.0
72.0
34.0
16.0
20.0
70.0
63.0
8.0
35.0
55.0
5.0
27.0
1.0
60.0
29.0
57.0
39.0
19.0
71.0
43.0
78.0
73.0
18.0
83.0
44.0
22.0
31.0
45.0
26.0
74.0
51.0
30.0
21.0
15.0
40.0
67.0
64.0
42.0
79.0
54.0
58.0
32.0
38.0
77.0
24.0
41.0
37.0
100.0
0.0
75.0
80.0
13.0
25.0


In [429]:
uniques_ingredients = df['ingredients'].unique()
uniques_ingredients_list = list(uniques_ingredients)
for value in uniques_ingredients_list:
    print(value)

fruit water sugar disaccharide salt addedsugar
empty
oilandfat sugar disaccharide addedsugar
sugar disaccharide addedsugar
oilandfat water addedsugar salt
water
flavouring oilandfat addedsugar
sugar disaccharide addedsugar salt

flavouring water sugar disaccharide addedsugar salt
salt
oilandfat
sugar disaccharide salt addedsugar
salt addedsugar
vegetable sugar disaccharide salt addedsugar
water salt
water oilandfat
oilandfat sugar disaccharide addedsugar salt
flavouring oilandfat sugar disaccharide addedsugar salt
flavouring sugar disaccharide addedsugar salt
water sugar disaccharide addedsugar
water naturalflavouring flavouring sugar disaccharide addedsugar
oilandfat flavouring water sugar disaccharide salt addedsugar
water dairy sugar disaccharide salt addedsugar
vegetableoilandfat oilandfat sugar disaccharide addedsugar salt
flavouring oilandfat water sugar disaccharide salt addedsugar
oilandfat water sugar disaccharide addedsugar salt
oilandfat salt
flavouring sugar disaccharide ad

In [430]:
uniques_categories = df['categories'].unique()
uniques_categories_list = list(uniques_categories)
for value in uniques_categories_list:
    print(value)


empty
sweetsnacks snacks
plantbasedfoods plantbasedfoodsandbeverages
meatsandtheirproducts
frozenfoods
dairies
snacks
meats meatsandtheirproducts
beverages plantbasedfoodsandbeverages
beverages
plantbasedfoodsandbeverages
cheeses fermentedmilkproducts fermentedfoods dairies
cerealsandpotatoes plantbasedfoods plantbasedfoodsandbeverages
spreads fruitsandvegetablesbasedfoods plantbasedfoods plantbasedfoodsandbeverages
cerealsandpotatoes snacks plantbasedfoods plantbasedfoodsandbeverages
beverages sweetsnacks snacks
condiments plantbasedfoods plantbasedfoodsandbeverages
condiments
beverages beveragesandbeveragespreparations
biscuitsandcakes sweetsnacks snacks
snacks plantbasedfoods plantbasedfoodsandbeverages
fruitsandvegetablesbasedfoods beverages plantbasedfoods plantbasedfoodsandbeverages
vegetablesbasedfoods fruitsandvegetablesbasedfoods plantbasedfoods plantbasedfoodsandbeverages
beverages plantbasedfoods plantbasedfoodsandbeverages
cocoaanditsproducts sweetsnacks snacks
confectione

In [431]:
uniques_labels_note = df['labels_note'].unique()
uniques_labels_note_list = list(uniques_labels_note)
for value in uniques_labels_note_list:
    print(value)

0.0
0.1111111111
0.2222222222
0.4444444444
0.3333333333
0.6666666667
1.0
0.8888888889
0.5555555556
0.7777777778


In [432]:
df.describe()

,groups,ecoscore_tags,ecoscore_score,countries,labels_note
count,164.000000,107.000000,500.000000,461.000000,500.000000
mean,0.395325,0.532710,48.446000,0.508058,0.070444
std,0.302127,0.297561,11.229572,0.344125,0.163465
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.111111,0.250000,49.000000,0.296089,0.000000
50%,0.444444,0.500000,49.000000,0.333333,0.000000
75%,0.666667,0.750000,49.000000,0.874317,0.000000
max,1.000000,1.000000,100.000000,1.000000,1.000000


In [433]:
def load_jsonl_to_dataframe(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    return df

def create_treemap_for_column(df, column_name):
    if column_name not in df.columns:
        print(f"warning, column: '{column_name}'does not exists")
        return
    data = df[column_name].value_counts().reset_index()
    data.columns = [column_name, 'count']
    fig = go.Figure(go.Treemap(
        labels=data[column_name],
        parents=[""] * len(data),
        values=data['count'],
        textinfo='label+value',  
    ))
    fig.update_layout(
        title=f'treemap column: {column_name}',
        paper_bgcolor='black',  
        plot_bgcolor='grey',   
        font_color='white'      
    )
    fig.show()

In [434]:
colonnes_a_travailler = ['ecoscore_tags', 'ecoscore_score', 'countries', 'labels_note']  
df = load_jsonl_to_dataframe(file_path)
for column in colonnes_a_travailler:
    create_treemap_for_column(df, column)

In [435]:
def count_unique_values(jsonl_file_path, colonnes_a_travailler):
    data = []
    with open(jsonl_file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    unique_counts = {}
    for column in colonnes_a_travailler:
        if column in df.columns:
            unique_count = df[column].nunique()
            if pd.api.types.is_numeric_dtype(df[column]):
                min_val = df[column].min()
                max_val = df[column].max()
                unique_counts[column] = {
                    'unique_count': unique_count,
                    'min': min_val,
                    'max': max_val
                }
            else:
                unique_counts[column] = {
                    'unique_count': unique_count
                }
        else:
            unique_counts[column] = 'Column not found in data'
    return unique_counts

In [436]:
unique_values = count_unique_values(file_path, colonnes_a_travailler)
print(unique_values)

{'ecoscore_tags': {'unique_count': 7, 'min': 0.0, 'max': 1.0}, 'ecoscore_score': {'unique_count': 102, 'min': 0.0, 'max': 100.0}, 'countries': {'unique_count': 888, 'min': 0.0, 'max': 1.0}, 'labels_note': {'unique_count': 10, 'min': 0.0, 'max': 1.0}}
